In [2]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
import data_extraction
import ARIMA
from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
past_days = 40
nRowsRead = None
grid_number = 1

In [5]:
def data_ANN(past_days):
    # input values
    # past_days is the number of past days of the grid_loss that the ANN will use
    loss_train_series = data_extraction.get_timeseries("train", nRowsRead, 'loss', grid_number)
    loss_test_series = data_extraction.get_timeseries("test", nRowsRead, 'loss', grid_number)
    load_train_series = data_extraction.get_timeseries("train", nRowsRead, 'load', grid_number)
    load_test_series = data_extraction.get_timeseries("test", nRowsRead, 'load', grid_number)
    
    loss_series = loss_train_series.append(loss_test_series)
    load_series = load_train_series.append(load_test_series)
    
    X = []
    y = []
    
    
    for i in range(len(loss_series) - past_days*24):
        X.append(loss_series[i:i+past_days*24:24].values + load_series[i:i+past_days*24:24].values)
        y.append(loss_series[i + past_days*24])
    
    return X, y



In [6]:
X,y = data_ANN(past_days)

In [7]:
train_size = len(data_extraction.get_timeseries("train", nRowsRead, 'loss', grid_number)) - past_days*24
X_train = X[:train_size]
y_train = y[:train_size]

X_test = X[train_size:]
y_test = y[train_size:]

In [8]:
model = MLPRegressor(solver="adam", 
                         alpha=1e-5, 
                         random_state=1, 
                         max_iter=20000,
                         tol=1e-4)

In [ ]:
model.get_params().keys()

In [ ]:
parameters = {'solver': ['adam'], 'hidden_layer_sizes' : [(i,i) for i in range(2,5,2)]}

In [ ]:
gsCV = GridSearchCV(model , parameters)

In [ ]:
gsCV.fit(X_train, y_train)

In [ ]:
sorted(gsCV.cv_results_.keys())

In [ ]:
gsCV.best_params_